# 3. Cassandra to Arrow

We use some code from the Cassandra server to read the SSTable, but instead of de/serializing to/from CQL, we use an [Arrow IPC stream](http://arrow.apache.org/), which is stored in a columnar format and better suited for analytics.

Data transformations:

1. SSTable on disk
2. Deserialized into Java Object in C* server
3. Client makes request to server (not to C* DB)
4. Data serialized via Arrow IPC stream
5. Sent across network
6. Arrow IPC stream received by client
7. Transformed into Arrow Table / cuDF

**Pros:**
- doesn't make request to the main Cassandra DB, which lessens the load and allows for other operations to run
- less de/serialization involved using the Arrow IPC stream

**Cons:**
- don't want to have to start Cassandra or use the JVM
- complex architecture

In [4]:
import pyarrow as pa
import pandas as pd
import socket

HOST = '127.0.0.1'
PORT = 9143

In [5]:
def read_bytes(sock, n):
    data = b''
    while len(data) < n:
        more = sock.recv(n - len(data))
        if not more:
            raise EOFError("Socket connection ended before reading specified number of bytes")
        data += more
    return data

def read_u8(sock):
    data = read_bytes(sock, 8)
    return int.from_bytes(data, byteorder='big')

# read data from socket
def fetch_data():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((HOST, PORT))
        sock.sendall(b'hello world\n')
        num_tables = read_u8(sock)
        table_buffers = []
        for i in range(num_tables):
            print('receiving table', i)
            table_size = read_u8(sock)
            buf = read_bytes(sock, table_size)
            table_buffers.append(buf)
    return table_buffers

In [47]:
buffers = fetch_data()
tables = [pa.ipc.open_stream(buf).read_all() for buf in buffers]
len(tables)

receiving table 0


1

In [48]:
tables[0].to_pandas()

,partition_key,_ts_row_liveness,_del_time_row_liveness,_ttl_row_liveness,_local_del_time_partition,_marked_for_del_at_partition,_local_del_time_row,_marked_for_delete_at_row,clustering_key_0,data,...,_del_time_data,_ttl_data,sensor_value,_ts_sensor_value,_del_time_sensor_value,_ttl_sensor_value,station_id,_ts_station_id,_del_time_station_id,_ttl_station_id
0,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.042,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.042,ollis quis risus sit amet venenatis. Suspendis...,...,NaT,NaT,94.353640,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
1,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.002,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.002,"ue sapien et, fermentum neque. Pellentesque mo...",...,NaT,NaT,95.759791,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
2,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.041,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.041,"Ut suscipit sem vel orci venenatis, a rutrum ...",...,NaT,NaT,86.207276,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
3,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.025,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.025,"tortor hendrerit, nec ultricies dui vestibulum...",...,NaT,NaT,97.209750,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
4,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.009,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.009,ctus mauris nec urna. Duis sit amet enim trist...,...,NaT,NaT,106.497951,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.018,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.018,"mi non consectetur pretium, diam augue maximus...",...,NaT,NaT,107.025525,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
4996,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.000,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.000,et odio a dolor placerat bibendum. Praesent au...,...,NaT,NaT,106.780531,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
4997,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.031,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.031,". Mauris vestibulum leo eu nunc commodo, at te...",...,NaT,NaT,100.584490,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
4998,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.014,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.014,"o, sollicitudin eget iaculis vitae, dapibus eu...",...,NaT,NaT,99.889286,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT


In [45]:
tables[1].to_pandas()

,partition_key,_ts_row_liveness,_del_time_row_liveness,_ttl_row_liveness,_local_del_time_partition,_marked_for_del_at_partition,_local_del_time_row,_marked_for_delete_at_row,count,_ts_count,_del_time_count,_ttl_count
0,test,2021-07-23 17:19:42.905355,1970-01-01 00:27:07.120782,0 days 16:40:00,NaT,NaT,NaT,NaT,13,NaT,NaT,NaT


In [46]:
tables[2].to_pandas()

,partition_key,_ts_row_liveness,_del_time_row_liveness,_ttl_row_liveness,_local_del_time_partition,_marked_for_del_at_partition,_local_del_time_row,_marked_for_delete_at_row
0,testlist,NaT,NaT,NaT,2021-07-23 17:23:42,2021-07-23 17:23:42.355027,NaT,NaT
